## Linear Regression


Sklearn based models are slightly finicky to get into a suitable onnx format. 
This notebook showcases how to do so using the `hummingbird-ml` python package ! 

In [4]:
!pip install hummingbird-ml

  Using cached hummingbird_ml-0.4.9-py2.py3-none-any.whl (165 kB)
  Using cached onnxconverter_common-1.14.0-py2.py3-none-any.whl (84 kB)
  Using cached scipy-1.11.4-cp39-cp39-macosx_12_0_arm64.whl (29.7 MB)
  Using cached scikit_learn-1.3.2-cp39-cp39-macosx_12_0_arm64.whl (9.5 MB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Using cached protobuf-3.20.2-py2.py3-none-any.whl (162 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.1
    Uninstalling protobuf-4.25.1:
      Successfully uninstalled protobuf-4.25.1
You should consider upgrading via the '/Users/ethancemer/Documents/development/ezkl/.env/bin/python -m pip install --upgrade pip' command.


In [5]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "hummingbird-ml"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass

import os
import torch
import ezkl
import json
from hummingbird.ml import convert


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
import numpy as np
from sklearn.linear_model import LinearRegression
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
# y = 1 * x_0 + 2 * x_1 + 3
y = np.dot(X, np.array([1, 2])) + 3
reg = LinearRegression().fit(X, y)
reg.score(X, y)

circuit = convert(reg, "torch", X[:1]).model

In [6]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')

witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [7]:
# export to onnx format
# !!!!!!!!!!!!!!!!! This will flash a warning but it is fine !!!!!!!!!!!!!!!!!!!!!

# Input to the model
shape = X.shape[1:]
# read in ./input_json
data = json.load(open("input.json", 'r'))
# convert to torch tensor
x = torch.tensor(data['input_data'], requires_grad=True)
torch_out = circuit(x)
# Export the model
torch.onnx.export(circuit,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

In [8]:
!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

res = ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True

In [9]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [10]:
# srs path
res = ezkl.get_srs(settings_path)

In [11]:
# now generate the witness file 

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [12]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

spawning module 2
spawning module 2


In [17]:
# GENERATE A PROOF
import time

proof_path = os.path.join('test.pf')
# log time it takes to generate proof
start = time.time()
res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        "single",
    )
end = time.time()
proving_time = end - start
print("PROOF GENERATION TIME: ", proving_time)

# define the path that stores the benchmarking results
benchmark_path = os.path.join('../../benchmarks.json')

# check that a benchmark path exists. If not, create one. Otherwise, load the existing one
if not os.path.isfile(benchmark_path):
    data = {
        "linear_regression": {
            "ezkl": {
                "provingTime": proving_time
            },
            "riscZero": {}
        }
    }
    with open(benchmark_path, 'w') as f:
        json.dump(data, open(benchmark_path, 'w'))
else:
    with open(benchmark_path, 'r') as f:
        benchmark = json.load(f)

    # Update the proving time in the loaded benchmark
    benchmark['linear_regression']['ezkl']['provingTime'] = proving_time

    # Write the updated benchmark back to the file
    with open(benchmark_path, 'w') as f:
        json.dump(benchmark, f, indent=4)


print(res)
print(res['instances'])
assert os.path.isfile(proof_path)

spawning module 2


PROOF GENERATION TIME:  0.04433393478393555


KeyError: 'ee'